# Satellite Constellation Crosslinking

## Create Scenario

### Be sure to download receiver and transmitter model, and paste file path string into the cell below

### Information regarding Solar and Lunar interference can be found in the _Graph & Report Manager_ "Gaps" report for both the constrained and unconstrained accesses.

In [ ]:
from win32api import GetSystemMetrics
import comtypes
#comtypes.client.gen_dir = None


import comtypes
from comtypes.client import CreateObject
print(comtypes.client.gen_dir)

#Configure laser Tx/Rx filenames
receiverfile = "File path for **VB_ReceiverModel.vbs**"
transmitterfile = "File path for **VB_TransmitterModel.vbs**"

#create instance of STK
App = CreateObject("STK12.Application")
App.Visible=True
App.UserControl=True

print(type(App))
#Grab handle on STK root
root = App.Personality2
App.WindowState = 2

In [ ]:
from comtypes.gen import STKUtil
from comtypes.gen import STKObjects
print(comtypes.client.gen_dir)

In [ ]:
#1. Create new scenario
root.NewScenario("CrossLinking_Scenario")
scenario = root.CurrentScenario

### Sets Analysis Time Period

In [ ]:
#from comtypes.gen import STKUtil
#from comtypes.gen import STKObjects

#2. Set analytical time period
scenario2 = scenario.QueryInterface(STKObjects.IAgScenario)
scenario2.SetTimePeriod('19 Jul 2020 12:00:000', '+365 days')
scenario2.Graphics.GndTracksVisible = False

#3. Reset animation time
root.Rewind();

## Generates the satellites, sensors, transmitters, and receivers

In [ ]:
numOrbitPlanes = 1
numSatsPerPlane = 17

for satNum in range(0,numSatsPerPlane): 
        
        #Insert satellite
        satellite = scenario.Children.New(STKObjects.eSatellite, f"SIREN{satNum+1}")
        satellite2 = satellite.QueryInterface(STKObjects.IAgSatellite)
                
        #Select Propagator
        satellite2.SetPropagatorType(STKObjects.ePropagatorJ4Perturbation)
        
        #Set initial state
        J4Propagator = satellite2.Propagator.QueryInterface(STKObjects.IAgVePropagatorJ4Perturbation)
        keplarian = J4Propagator.InitialState.Representation.ConvertTo(STKUtil.eOrbitStateClassical).QueryInterface(STKObjects.IAgOrbitStateClassical)
        
        keplarian.SizeShapeType = STKObjects.eSizeShapeSemimajorAxis
        keplarian.SizeShape.QueryInterface(STKObjects.IAgClassicalSizeShapeSemimajorAxis).SemiMajorAxis = 6953.1363 #km
        keplarian.SizeShape.QueryInterface(STKObjects.IAgClassicalSizeShapeSemimajorAxis).Eccentricity = 0

        keplarian.Orientation.Inclination = 60 #degrees
        keplarian.Orientation.ArgOfPerigee = 0 #degrees
        
        keplarian.LocationType = STKObjects.eLocationTrueAnomaly
        keplarian.Location.QueryInterface(STKObjects.IAgClassicalLocationTrueAnomaly).Value = 0 + (360/numSatsPerPlane)*satNum  
        
        #Propagate
        satellite2.Propagator.QueryInterface(STKObjects.IAgVePropagatorJ4Perturbation).InitialState.Representation.Assign(keplarian)
        satellite2.Propagator.QueryInterface(STKObjects.IAgVePropagatorJ4Perturbation).Propagate()
        
        #Attaches and configures a forward-looking sensor
        Forw = satellite.Children.New(STKObjects.eSensor,f"Prograde_{satNum}")
        Forw2 = Forw.QueryInterface(STKObjects.IAgSensor)
        
        Forw2.CommonTasks.SetPatternSimpleConic(5, 2)
        Forw2.SetPointingType(5)
        
        Forw2.VO.PercentTranslucency = 80
        ForwAtt = Forw2.Graphics.QueryInterface(STKObjects.IAgSnGraphics)
        ForwAtt.Color = 102204
        
        #Attaches and configures a forward-looking receiver
        ForwObj = Forw.QueryInterface(STKObjects.IAgStkObject)
        receiverf = ForwObj.Children.New(17, f"ProgradeReceiver_{satNum}")
        receiverf2 = receiverf.QueryInterface(STKObjects.IAgReceiver)
        receiverf2.SetModel('Script Plugin Laser Receiver Model')
        recmodelf = receiverf2.Model
        recmodelf2 = recmodelf.QueryInterface(STKObjects.IAgReceiverModelScriptPlugin)
        recmodelf2.Filename = receiverfile
        
        #Attaches and configures a forward-looking transmitter
        txforward = ForwObj.Children.New(24, f"ProgradeTransmitter_{satNum}")
        txforward2 = txforward.QueryInterface(STKObjects.IAgTransmitter)
        txforward2.SetModel('Script Plugin Laser Transmitter Model')
        txforwardmodel = txforward2.Model
        txforwardmodel2 = txforwardmodel.QueryInterface(STKObjects.IAgTransmitterModelScriptPlugin)
        txforwardmodel2.Filename = transmitterfile
        
        #Attaches and configures a backward-looking sensor
        Back = satellite.Children.New(STKObjects.eSensor,f"Retrograde_{satNum}")
        Back2 = Back.QueryInterface(STKObjects.IAgSensor)
        
        Back2.CommonTasks.SetPatternSimpleConic(5, 2)
        Back2.SetPointingType(5)
        
        Back2.VO.PercentTranslucency = 80
        BackAtt = Back2.Graphics.QueryInterface(STKObjects.IAgSnGraphics)
        BackAtt.Color = 204
        
        #Attaches and configures a backward-looking receiver
        BackObj = Back.QueryInterface(STKObjects.IAgStkObject)
        receiverb = BackObj.Children.New(17, f"RetrogradeReceiver_{satNum}")
        receiverb2 = receiverb.QueryInterface(STKObjects.IAgReceiver)
        receiverb2.SetModel('Script Plugin Laser Receiver Model')
        recmodelb = receiverb2.Model
        recmodelb2 = recmodelb.QueryInterface(STKObjects.IAgReceiverModelScriptPlugin)
        recmodelb2.Filename = receiverfile
        
        #Attaches and configures a backward-looking transmitter
        txbackward = BackObj.Children.New(24, f"RetrogradeTransmitter_{satNum}")
        txbackward2 = txbackward.QueryInterface(STKObjects.IAgTransmitter)
        txbackward2.SetModel('Script Plugin Laser Transmitter Model')
        txbackwardmodel = txbackward2.Model
        txbackwardmodel2 = txbackwardmodel.QueryInterface(STKObjects.IAgTransmitterModelScriptPlugin)
        txbackwardmodel2.Filename = transmitterfile

## Iterates through all satellites and configures proper pointing

In [ ]:
SatList = scenario.Children.GetElements(STKObjects.eSatellite)
for satellite in range(SatList.Count):
    SensorList = SatList[satellite].Children.GetElements(STKObjects.eSensor)
    Pro = SensorList[0].QueryInterface(STKObjects.IAgSensor)
    Ret = SensorList[1].QueryInterface(STKObjects.IAgSensor)
    
    
    ProTargeting = Pro.Pointing.QueryInterface(STKObjects.IAgSnPointing)
    ProTar = ProTargeting.QueryInterface(STKObjects.IAgSnPtTargeted).Targets
    
    RetTargeting = Ret.Pointing.QueryInterface(STKObjects.IAgSnPointing)
    RetTar = RetTargeting.QueryInterface(STKObjects.IAgSnPtTargeted).Targets
    
    end = SatList.Count - 1
    
    if satellite == 0:
        ProTar.AddObject(SatList[1])
        RetTar.AddObject(SatList[end])
        
    elif satellite == (end):
        ProTar.AddObject(SatList[0])
        RetTar.AddObject(SatList[end-1])
        
    else:
        ProTar.AddObject(SatList[satellite+1])
        RetTar.AddObject(SatList[satellite-1])

## Creates Solar and Lunar Exclusion Angle Constraints

In [ ]:
progradeTransmitters = []
retrogradeReceivers = []
retrogradeTransmitters = []
progradeReceivers = []
for satellite in range(SatList.Count):
    SensorList = SatList[satellite].Children.GetElements(STKObjects.eSensor)
    Pro = SensorList[0].QueryInterface(STKObjects.IAgSensor)
    Ret = SensorList[1].QueryInterface(STKObjects.IAgSensor)
    progradeTransmitter = Pro.QueryInterface(STKObjects.IAgStkObject).Children.GetElements(STKObjects.eTransmitter)[0]
    progradeTransmitters.append(progradeTransmitter)
    retrogradeReceiver = Ret.QueryInterface(STKObjects.IAgStkObject).Children.GetElements(STKObjects.eReceiver)[0]
    retrogradeReceivers.append(retrogradeReceiver)
    progradeReceiver = Pro.QueryInterface(STKObjects.IAgStkObject).Children.GetElements(STKObjects.eReceiver)[0]
    progradeReceivers.append(progradeReceiver)
    retrogradeTransmitter = Ret.QueryInterface(STKObjects.IAgStkObject).Children.GetElements(STKObjects.eTransmitter)[0]
    retrogradeTransmitters.append(retrogradeTransmitter)
    
for index in range(len(retrogradeReceivers)):
    if index == 0:
        proTrans = progradeTransmitters[-1].QueryInterface(STKObjects.IAgStkObject)
    else:
        proTrans = progradeTransmitters[index-1].QueryInterface(STKObjects.IAgStkObject)
    retRec = retrogradeReceivers[index].QueryInterface(STKObjects.IAgStkObject)
    retTrans = retrogradeTransmitters[index].QueryInterface(STKObjects.IAgStkObject)
    if index == 0:
        proRec = progradeReceivers[-1].QueryInterface(STKObjects.IAgStkObject)
    else:
        proRec = progradeReceivers[index-1].QueryInterface(STKObjects.IAgStkObject)
    accConstr = proTrans.AccessConstraints
    cnstrSunAngle = accConstr.AddConstraint(29)
    cnstrSunAngle2 = cnstrSunAngle.QueryInterface(STKObjects.IAgAccessCnstrAngle)
    cnstrSunAngle2.Angle = 4
    cnstrMoonAngle = accConstr.AddConstraint(28)
    cnstrMoonAngle2 = cnstrMoonAngle.QueryInterface(STKObjects.IAgAccessCnstrAngle)
    cnstrMoonAngle2.Angle = 4
    if index == 0:
        access = proTrans.GetAccessToObject(retRec)
        access.ComputeAccess()
        unconstrainedAccess = retTrans.GetAccessToObject(proRec)
        unconstrainedAccess.ComputeAccess()